In [6]:
import numpy as np

In [99]:
class Layer:
    def __init__(self, requires_grad=True):
        self.requires_grad = requires_grad

    def apply(self, state, qaoa, angle):
        raise NotImplementedError("Subclasses should implement this method.")

class MixerLayer(Layer):
    def __init__(self, requires_grad=True):
        self.requires_grad = requires_grad
        self.mixer = None
    def apply(self, state, qaoa, angle):
        if self.mixer == None or len(self.mixer) != qaoa.n_qubits:
            self.mixer = mixer_list(qaoa.n_qubits)
        return apply_beta(qaoa.n_qubits, self.mixer, angle, state)

class CostLayer(Layer):
    def __init__(self, requires_grad=True):
        self.requires_grad = requires_grad
    def apply(self, state, qaoa, angle):
        return apply_gamma(qaoa, angle, state)

def mixer_list(n_qubits):
    """Generates a list of indices for state vector swapping in the mixer.
    Returns:
        List: x_list  - List of lists of indices.
    """             
    def split(x, k):
        return x.reshape((2**k, -1))
    
    def sym_swap(x):
        return np.asarray([x[-1], x[-2], x[1], x[0]])

    x_list = []
    t1 = np.asarray([np.arange(2**(n_qubits-1), 2**n_qubits), np.arange(0, 2**(n_qubits-1))])
    t1 = t1.flatten()
    x_list.append(t1.flatten())
    t2 = t1.reshape(4, -1)
    t3 = sym_swap(t2)
    t1 = t3.flatten()
    x_list.append(t1)
    k = 1
    
    while k < (n_qubits - 1):
        t2 = split(t1, k)
        t2 = np.asarray(t2)
        t1 = []
        for y in t2:
            t3 = y.reshape((4, -1))
            t4 = sym_swap(t3)
            t1.append(t4.flatten())
        t1 = np.asarray(t1)
        t1 = t1.flatten()
        x_list.append(t1)
        k += 1
    
    return x_list
    
def apply_gamma(qaoa, gamma: float, statevector:  np.ndarray) ->  np.ndarray:
    return  np.exp(-1j * gamma * qaoa.H) * statevector

def apply_beta(n_qubits, x_list,  beta: float, statevector: np.ndarray) ->  np.ndarray:
    c = np.cos(beta)
    s = np.sin(beta)
    statevector_new = np.copy(statevector)
    for i in range(n_qubits):
        statevector_swap = statevector_new[x_list[i]]
        statevector_new = -1j * s * statevector_swap + c * statevector_new
    return statevector_new
    
class IdentityLayer(Layer):
    def __init__(self, requires_grad=False):
        self.requires_grad = requires_grad
    def apply(self, state, qaoa, angle):
        return state


In [100]:
import numpy as np

class QAOA:
    def __init__(self, layers, H):
        self.layers = layers
        self.H = H
        self.n_qubits = int(np.log2(len(H)))

        # Initialize angles as random values
        self.angles = np.random.uniform(0, 2*np.pi, size=(len(self.layers)))

    def plus_state(self):
        d = 2**self.n_qubits
        return np.array([1/np.sqrt(d)]*d, dtype='complex128')

    def forward(self, initial_state):
        state = initial_state
        for layer, angle in zip(self.layers, self.angles):
            state = layer.apply(state=state, qaoa=self, angle=angle)
        return state

    def expectation(self):
        initial_state = self.plus_state()
        final_state = self.forward(initial_state)
        print(final_state)
        ex = np.real(np.vdot(final_state, self.H * final_state))
        return ex


In [343]:
# Example Hamiltonian
H = np.array([1,1,1,0])

# Define the layers
layers = [
    MixerLayer(),
    CostLayer(),
    # MixerLayer(),
    # CostLayer(),
    # IdentityLayer(),
]

# Create QAOA instance
qaoa = QAOA(layers, H)


# Calculate expectation value
print(qaoa.expectation())

[-0.48991363-0.09992313j -0.48991363-0.09992313j -0.48991363-0.09992313j
  0.11460433+0.48668865j]
0.7499999999999999
